In [1]:
# Data Manipulating and Visualization
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
import random


# Operating System
import os
from datetime import datetime

# Machine Learning Algorithms
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

# Performance metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

# Hyperparameter
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

## Mathematics and Statistics
import scipy.stats as stats
from scipy.stats import uniform
from scipy.stats import loguniform

# NLP related 
import string
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Load modeling data
### [The creation of the the modeling data is discussed in this notebook](https://github.com/houzhj/Machine_Learning/blob/main/ipynb/imdb_data.ipynb)

In [2]:
Modeling_Date = pd.read_csv('/kaggle/input/imdb-date/Modeling_Date.csv',low_memory=False)
X_Data = Modeling_Date.drop(['movie_review','sentiment_label','sentiment_number'], axis=1)
Y_Data = Modeling_Date.sentiment_number

### First, build a model with all the 2000 features. 

In [3]:
x1,x2,y1,y2 = train_test_split(X_Data,Y_Data,test_size = 0.3,random_state = 42)

params_LGB= {'boosting_type'    : 'gbdt',
             'objective'        : 'binary',
             'colsample_bytree' : 0.8,
             'learning_rate'    : 0.05,
             'min_child_samples': 10,
             'min_child_weight' : 5,
             'max_depth'        : -1,
             'min_split_gain'   : 0,
             'num_leaves'       : 31,
             'subsample_for_bin': 50000,
             'n_estimators'     : 5000,
             'subsample_freq'   : 1
}

LGB_2000 = lgb.LGBMClassifier(**params_LGB,importance_type='gain')
LGB_2000.fit(X = x1, y = y1,
             eval_metric=['auc','logloss'], eval_set=[(x1,y1),(x2,y2)],
             callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])

importance_df = pd.DataFrame(list(x1)).rename(columns={0:'Features'})
importance_df['importance'] = LGB_2000.feature_importances_
importance_df = importance_df.sort_values(by=['importance'],ascending=False).reset_index(drop=True)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[838]	training's auc: 0.996931	training's binary_logloss: 0.127731	valid_1's auc: 0.944385	valid_1's binary_logloss: 0.301285


### There are 1520 features with postive feature importance. 

In [4]:
importance_df[importance_df['importance']>0]

,Features,importance
0,bad,26179.362509
1,worst,20654.644251
2,wast,17914.736233
3,great,11337.439661
4,aw,9731.041632
...,...,...
1515,luck,2.224650
1516,univers,2.213020
1517,fred,2.198200
1518,dub,2.010700


### To reduce the running time, I am keeping only the top 500 features among these 2000 features in the model, based on their importance. Then conduct the feature selection analysis (below) from these 500 features. 

In [5]:
##### Keep the top 500 features
keep_features = importance_df.head(500)['Features']

In [6]:
X_Data_top500 = X_Data[keep_features]

# 2. Create Random Noise Features

In [7]:
def create_random_noises(n_random_noise,df):
    new_df = df.copy()
    n_rows = len(df)
    noise_features = []
    
    ##### Adding random noises that follow Normal(0,1) distribution
    for i in range(n_random_noise):
        new_df['norm_'+str(i+1)] = np.random.normal(0,1,n_rows)
        noise_features = noise_features + ['norm_'+str(i+1)]
    
    ##### Adding random noises that follow Normal(0,1) distribution
    for i in range(n_random_noise):
        new_df['unif_'+str(i+1)] = np.random.uniform(0,1,n_rows)
        noise_features = noise_features + ['unif_'+str(i+1)]
    ##### Adding random noises that follow Bernoulli(0.5) distribution
    for i in range(n_random_noise):
        new_df['ber_'+str(i+1)] = np.random.binomial(1,0.5,n_rows)
        noise_features = noise_features + ['ber_'+str(i+1)] 
        
    return new_df,noise_features

### This is how the function works.
- #### There are three distributions: Normal(0,1), Uniform(0,1), and Bernoulli(0.5). These are fixed. 
- #### For exampel, if n_random_noise = 3, the function creates 3 random noise features for each of the distributions. 
- #### These features are added to the original dataframe, with all the original columns unchanged. 

In [8]:
temp = pd.DataFrame( {'A': list(range(10)),
        'B': list(range(10,20)),
        'C': list(range(20,30))})
create_random_noises(3,temp)

(   A   B   C    norm_1    norm_2    norm_3    unif_1    unif_2    unif_3  \
 0  0  10  20 -0.626849 -0.991432 -0.662590  0.975370  0.029285  0.604849   
 1  1  11  21  0.153650  0.178153  0.418251  0.177928  0.637209  0.066638   
 2  2  12  22  0.206623  1.916912 -0.282340  0.029553  0.269940  0.069398   
 3  3  13  23 -0.437213 -0.440159 -0.449994  0.054007  0.273678  0.062441   
 4  4  14  24 -0.838972  0.160270  0.032290  0.920138  0.085556  0.784156   
 5  5  15  25  0.198322  0.979394 -0.624693  0.622266  0.672422  0.555796   
 6  6  16  26  1.464103 -0.489683  0.255679  0.803987  0.439981  0.174949   
 7  7  17  27  1.168045  0.450790  0.434817  0.583797  0.263613  0.567969   
 8  8  18  28 -1.035023  0.404480  0.057958  0.895280  0.023530  0.328578   
 9  9  19  29  0.116945  2.024884 -0.503333  0.183130  0.174395  0.133494   
 
    ber_1  ber_2  ber_3  
 0      1      0      1  
 1      0      1      0  
 2      1      1      1  
 3      1      0      0  
 4      0      1     

### Apply this function to X_Data (the dataframe for predictive features)

In [9]:
X_Data_with_noise,noise_feature_names = create_random_noises(5,X_Data_top500)

In [10]:
X_Data_with_noise[noise_feature_names].describe()

,norm_1,norm_2,norm_3,norm_4,norm_5,unif_1,unif_2,unif_3,unif_4,unif_5,ber_1,ber_2,ber_3,ber_4,ber_5
count,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000,49582.000000
mean,0.001828,0.001218,-0.007725,-0.001759,-0.009726,0.500396,0.499292,0.500662,0.502463,0.500052,0.500766,0.499839,0.500262,0.501936,0.496975
std,0.996881,1.000142,1.000060,0.994839,0.998677,0.289390,0.289421,0.289109,0.288868,0.288815,0.500004,0.500005,0.500005,0.500001,0.499996
min,-4.053176,-4.593791,-4.530292,-4.140811,-4.245464,0.000021,0.000012,0.000036,0.000009,0.000038,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.670963,-0.675347,-0.680455,-0.668804,-0.685493,0.248966,0.245926,0.249252,0.252811,0.250181,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.001923,-0.000599,-0.007004,-0.004916,-0.009682,0.502668,0.499558,0.499783,0.502308,0.501425,1.000000,0.000000,1.000000,1.000000,0.000000
75%,0.671993,0.677425,0.671397,0.674417,0.660919,0.751648,0.750786,0.752307,0.753162,0.749737,1.000000,1.000000,1.000000,1.000000,1.000000
max,3.762298,3.959252,4.132506,3.765039,4.230507,0.999989,0.999993,0.999971,0.999975,0.999985,1.000000,1.000000,1.000000,1.000000,1.000000


# 3. Random Noise Test
### The random noise features do not have any intuitions, and therefore should not be selected. The goal of the random noise test is to evaluate a feature selection method, according to whether the method is able to identify these random noise feauture, and not to select them.
### According to the results below, 
- #### The Iterative Reduction approach is not able to identify random noise features that follow N(0,1) and Uniform[0,1] distributions.
- #### The Boruta approach works better at identifying the random noise features. 

## 3.1 Iterative Reduction
### See introduction about the Iterative Reduction [in this notebook](https://github.com/houzhj/Machine_Learning/blob/main/ipynb/imdb_featureselection.ipynb). 
### The following codes perform the IR approach.

In [11]:
def Run_IR(params_LGB,importance_type,x1,x2,y1,y2,n_feature_list):
    def get_importance_from_LGBM(model,x_data,importance_type):
        ### Get the feature importance list, merge with the columns names, and sorted by the importance score. 
        importance_df = pd.DataFrame(list(x_data)).rename(columns={0:'Features'})
        importance_df[importance_type] = model.feature_importances_
        importance_df = importance_df.sort_values(by=[importance_type],ascending=False)
        return importance_df

    def iterative_reduction(start_n,end_n,importance_dict):
        print('Feature reduction from %d to %d ...' %(start_n,end_n))
        importance_old = importance_dict['importance_'+str(start_n)]
        Selected_Features = importance_old.head(end_n)['Features'].values.tolist()
        Shuffled_Features = Selected_Features.copy()
        random.shuffle(Shuffled_Features)
        x1_new = x1[Shuffled_Features]
        x2_new = x2[Shuffled_Features]
        
        LGB_now = lgb.LGBMClassifier(**params_LGB,importance_type=importance_type)
        LGB_now.fit(X = x1_new, y = y1,
                    eval_metric=['logloss'], eval_set=[(x1_new,y1),(x2_new,y2)],
                    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
        importance_now = get_importance_from_LGBM(LGB_now,x1_new,importance_type)
        
        updated_importance_dict = importance_dict.copy()
        updated_importance_dict['importance_'+str(end_n)] = importance_now
        return(updated_importance_dict)

    ##### Create a dict to store the importance list
    importance_dict   = {}
    
    ##### First iteration (including all the features considered)
    LGB_515 = lgb.LGBMClassifier(**params_LGB,importance_type=importance_type)
    LGB_515.fit(X = x1, y = y1,
                 eval_metric=['auc','logloss'], eval_set=[(x1,y1),(x2,y2)],
                 callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
    importance_515 = get_importance_from_LGBM(LGB_515,x1,importance_type)
    importance_dict['importance_515'] = importance_515
    
    ##### Run Iteration Reduction
    for a in range(len(n_feature_list)-1):
        importance_dict = iterative_reduction(n_feature_list[a], n_feature_list[a+1],importance_dict)
    
    ##### Create a dict to store the feature list
    feature_list_dict = {}
    for i in range(len(n_feature_list)):
        temp = importance_dict['importance_'+str(n_feature_list[i])]
        feature_list_dict['fl_ir_'+str(n_feature_list[i])] = temp['Features']
    
    return feature_list_dict

In [12]:
n_feature_list   =  [515]+[500-i*20 for i in range(20)]
x1,x2,y1,y2 = train_test_split(X_Data_with_noise,Y_Data,test_size = 0.3,random_state = 42)

In [13]:
IR_fl = Run_IR(params_LGB,'gain',x1,x2,y1,y2,n_feature_list)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[626]	training's auc: 0.991925	training's binary_logloss: 0.162885	valid_1's auc: 0.940761	valid_1's binary_logloss: 0.310783
Feature reduction from 515 to 500 ...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[848]	training's binary_logloss: 0.13375	valid_1's binary_logloss: 0.312329
Feature reduction from 500 to 480 ...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[755]	training's binary_logloss: 0.145717	valid_1's binary_logloss: 0.311656
Feature reduction from 480 to 460 ...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[684]	training's binary_logloss: 0.15587	valid_1's binary_logloss: 0.312514
Feature reduction from 460 to 440 ...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[678]	trainin

In [14]:
fl_names = ['fl_ir_'+ str(i) for i in n_feature_list]
rank_ir_df = pd.DataFrame(columns=['feature_list','n_features']+noise_feature_names)
for fl in range(len(fl_names)):
    fl_now = list(IR_fl[fl_names[fl]])
    rank_ir_df.loc[fl,'feature_list'] = fl_names[fl]
    rank_ir_df.loc[fl,'n_features']   = len(fl_now)
    for noise_name in noise_feature_names:
        if noise_name in fl_now: 
            rank_ir_df.loc[fl,noise_name] = fl_now.index(noise_name)+1
        else:
            rank_ir_df.loc[fl,noise_name] = np.nan

rank_ir_df

,feature_list,n_features,norm_1,norm_2,norm_3,norm_4,norm_5,unif_1,unif_2,unif_3,unif_4,unif_5,ber_1,ber_2,ber_3,ber_4,ber_5
0,fl_ir_515,515,107,101,142,106,112,135,99,93,86,111,511,509,512,514,515
1,fl_ir_500,500,80,64,87,59,74,84,63,62,56,65,NaN,NaN,NaN,NaN,NaN
2,fl_ir_480,480,94,72,85,64,86,96,75,89,65,82,NaN,NaN,NaN,NaN,NaN
3,fl_ir_460,460,98,97,95,88,105,111,93,77,73,90,NaN,NaN,NaN,NaN,NaN
4,fl_ir_440,440,104,92,103,75,101,116,74,78,60,95,NaN,NaN,NaN,NaN,NaN
5,fl_ir_420,420,117,108,138,100,121,153,114,106,99,112,NaN,NaN,NaN,NaN,NaN
6,fl_ir_400,400,94,80,108,86,103,114,92,88,78,97,NaN,NaN,NaN,NaN,NaN
7,fl_ir_380,380,103,96,133,90,112,157,102,98,69,97,NaN,NaN,NaN,NaN,NaN
8,fl_ir_360,360,98,122,112,110,104,120,94,87,80,97,NaN,NaN,NaN,NaN,NaN
9,fl_ir_340,340,108,96,121,116,117,151,102,89,97,110,NaN,NaN,NaN,NaN,NaN


### Observations 
#### - Among the 15 random noise features, all of the Normal noises and the Uniform noises are relatively important. 
#### - Through the IR procedure, their feature importance ranks relatively high, within the top 100. Therefore, the IR approach cannot successfully remove them.

## 3.2 Boruta
### See introduction about the Boruta [in this notebook](https://github.com/houzhj/Machine_Learning/blob/main/ipynb/imdb_featureselection.ipynb). 
### The following codes perform the Boruta approach.

In [15]:
def Run_Boruta(params_LGB,importance_type,X_Data,Y_Data,TH_values,N_iteration):
    hits            = np.zeros(len(X_Data.columns))
    early_stopping_rounds = 50
    for iter_ in range(N_iteration):
        np.random.seed(iter_)
        X_shadow = X_Data.apply(np.random.permutation)
        X_shadow.columns = ['shadow'+ feature for feature in X_Data.columns]
        X_boruta = pd.concat([X_Data,X_shadow],axis=1,ignore_index=True)
        x_train,x_valid,y_train,y_valid = train_test_split(X_boruta,
                                                            Y_Data,
                                                            test_size = 0.3, 
                                                            random_state = 100)
        LGB_model = lgb.LGBMClassifier(**params_LGB,importance_type=importance_type)
        LGB_model.fit(X = x_train, y = y_train,
                     eval_metric=['auc','logloss'], eval_set=[(x_train,y_train),(x_valid,y_valid)],
                     callbacks=[lgb.early_stopping(early_stopping_rounds), lgb.log_evaluation(0)])
        feature_imp_X      = LGB_model.feature_importances_[:len(X_Data.columns)]
        feature_imp_shadow = LGB_model.feature_importances_[len(X_Data.columns):]
        hits+=(feature_imp_X>feature_imp_shadow.max())
    
    Hits_df = pd.DataFrame(columns=['Feature','Hits'])
    Hits_df.iloc[:,0] = X_Data.columns
    Hits_df.iloc[:,1] = hits
    
    feature_list_dict = {}
    for i in range(len(TH_values)):
        feature_keep = Hits_df[Hits_df['Hits']>=TH_values[i]]['Feature']
        feature_list_dict['fl_boruta_'+str(TH_values[i])] = feature_keep
    return feature_list_dict

In [16]:
TH_values=[1,2,3,4,5,6,7,8,9,10,20,30,40,50]
Boruta_fl = Run_Boruta(params_LGB,'gain',X_Data_with_noise,Y_Data,TH_values,50)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[831]	training's auc: 0.997729	training's binary_logloss: 0.127295	valid_1's auc: 0.942126	valid_1's binary_logloss: 0.306996
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[601]	training's auc: 0.993025	training's binary_logloss: 0.162839	valid_1's auc: 0.942198	valid_1's binary_logloss: 0.307753
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[633]	training's auc: 0.994177	training's binary_logloss: 0.156448	valid_1's auc: 0.942195	valid_1's binary_logloss: 0.307544
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[744]	training's auc: 0.9966	training's binary_logloss: 0.138872	valid_1's auc: 0.942006	valid_1's binary_logloss: 0.307496
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[561]	training's a

In [18]:
fl_names = ['fl_boruta_'+ str(i) for i in TH_values]
noise_boruta_df = pd.DataFrame(columns=['feature_list','n_features']+noise_feature_names)
for fl in range(len(fl_names)):
    fl_now = list(Boruta_fl[fl_names[fl]])
    noise_boruta_df.loc[fl,'feature_list'] = fl_names[fl]
    noise_boruta_df.loc[fl,'n_features']   = len(fl_now)
    for noise_name in noise_feature_names:
        if noise_name in fl_now: 
            noise_boruta_df.loc[fl,noise_name] = 1
        else:
            noise_boruta_df.loc[fl,noise_name] = 0
noise_boruta_df

,feature_list,n_features,norm_1,norm_2,norm_3,norm_4,norm_5,unif_1,unif_2,unif_3,unif_4,unif_5,ber_1,ber_2,ber_3,ber_4,ber_5
0,fl_boruta_1,187,0,1,0,1,0,0,1,1,1,1,0,0,0,0,0
1,fl_boruta_2,179,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0
2,fl_boruta_3,178,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
3,fl_boruta_4,166,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
4,fl_boruta_5,162,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
5,fl_boruta_6,159,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
6,fl_boruta_7,156,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
7,fl_boruta_8,155,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
8,fl_boruta_9,149,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
9,fl_boruta_10,147,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


### Observations 
#### - Among the 15 random noise features, most of them only obtain very few hits (i.e., being more predictive than all the shadow features). 
#### - They can be easily removed, even when the threshold for the number of hits is set very low. 
#### - Three random features obtain 10 or more hits, with one being normal noise and two being uniform noise.